In [1]:
import json
import numpy as np
from collections import OrderedDict
import pandas as pd
import plotly.express as px
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import base64_thermal_decoder as decoder

### Reading and combining the data

In [2]:
# Load samples from json file
with open('samples.json', 'r') as f:
    samples = json.load(f)
# Decode base64 strings
decoded_samples = OrderedDict()
for timestamp, encoded_string in samples.items():
    values = decoder.decode_base64(encoded_string, 0, 100.0)
    decoded_samples[timestamp] = np.array(values, dtype=np.float32).reshape(24, 32, 1)
# Load labels from json file
with open('labels.json', 'r') as f:
    labels = json.load(f)
# Set sample labels
labeled_samples = {}
actual_label_idx = 0
actual_label_start = labels[actual_label_idx][0]
actual_label_end = labels[actual_label_idx][1]
actual_label_name = labels[actual_label_idx][2]
for timestamp, _ in decoded_samples.items():
    if actual_label_start <= int(timestamp) <= actual_label_end:
        labeled_samples[timestamp] = {
            'data': decoded_samples[timestamp],
            'label': actual_label_name
        }
    else:
        labeled_samples[timestamp] = {
            'data': decoded_samples[timestamp],
            'label': None
        }
        if int(timestamp) > actual_label_end:
            # Go to next label until the end of the labels
            if actual_label_idx < len(labels) - 1:
                actual_label_idx += 1
                actual_label_start = labels[actual_label_idx][0]
                actual_label_end = labels[actual_label_idx][1]
                actual_label_name = labels[actual_label_idx][2]

In [3]:
df = pd.DataFrame.from_dict(labeled_samples, orient='index')
df.head()

,data,label
1703022685733,"[[[19.12], [20.41], [22.69], [23.46], [23.85],...",None
1703022687761,"[[[18.78], [20.62], [22.89], [23.88], [23.51],...",None
1703022689790,"[[[18.97], [21.24], [22.84], [23.64], [24.06],...",None
1703022691819,"[[[19.19], [20.9], [23.04], [23.51], [24.08], ...",None
1703022693849,"[[[19.26], [20.65], [23.35], [23.9], [23.78], ...",None


In [4]:
df['label'].value_counts()

DD     119
DLD     39
DLE     36
Name: label, dtype: int64

### Preparing the data

In [5]:
df.dropna(inplace=True)
X = np.array(df['data'])
y = np.array(df['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [6]:
unique, counts = np.unique(y_train, return_counts=True)
print('y_train: ', dict(zip(unique, counts)))
unique, counts = np.unique(y_test, return_counts=True)
print('y_test: ', dict(zip(unique, counts)))

y_train:  {'DD': 70, 'DLD': 24, 'DLE': 22}
y_test:  {'DD': 49, 'DLD': 15, 'DLE': 14}


In [7]:
y_train = LabelEncoder().fit_transform(y_train)
y_test = LabelEncoder().fit_transform(y_test)

unique, counts = np.unique(y_train, return_counts=True)
print('y_train: ', dict(zip(unique, counts)))
unique, counts = np.unique(y_test, return_counts=True)
print('y_test: ', dict(zip(unique, counts)))

X_train = np.array([x.reshape(24, 32) for x in X_train])
X_test = np.array([x.reshape(24, 32) for x in X_test])

y_train:  {0: 70, 1: 24, 2: 22}
y_test:  {0: 49, 1: 15, 2: 14}


In [8]:
# Convert to tensors
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

### Defining the model

In [9]:
model = tf.keras.Sequential(
    [
        layers.InputLayer(input_shape=(24, 32)),
        layers.Reshape(target_shape=(24, 32, 1)),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(3, activation='softmax')
    ]
)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 32, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 22, 30, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 13, 64)         18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 4, 128)         7

In [10]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,
    mode='min',
    restore_best_weights=True
)

### Training the model

In [11]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping]
)

Epoch 1/100
4/4 [==============================] - 2s 176ms/step - loss: 1.4060 - accuracy: 0.5690 - val_loss: 1.2072 - val_accuracy: 0.1923
Epoch 2/100
4/4 [==============================] - 0s 43ms/step - loss: 1.2132 - accuracy: 0.4310 - val_loss: 0.9289 - val_accuracy: 0.6282
Epoch 3/100
4/4 [==============================] - 0s 32ms/step - loss: 1.0230 - accuracy: 0.6466 - val_loss: 0.9098 - val_accuracy: 0.6282
Epoch 4/100
4/4 [==============================] - 0s 31ms/step - loss: 0.9357 - accuracy: 0.6034 - val_loss: 0.9173 - val_accuracy: 0.6282
Epoch 5/100
4/4 [==============================] - 0s 31ms/step - loss: 0.9499 - accuracy: 0.6034 - val_loss: 0.8837 - val_accuracy: 0.6282
Epoch 6/100
4/4 [==============================] - 0s 31ms/step - loss: 0.9491 - accuracy: 0.6034 - val_loss: 0.8916 - val_accuracy: 0.6282
Epoch 7/100
4/4 [==============================] - 0s 33ms/step - loss: 0.9493 - accuracy: 0.6034 - val_loss: 0.8812 - val_accuracy: 0.6282
Epoch 8/100
4/4 [==

In [12]:
px.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'},
    title='Training and validation loss'
)

In [13]:
px.line(
    history.history,
    y=['accuracy', 'val_accuracy'],
    labels={'index': 'epoch', 'value': 'accuracy'},
    title='Training and validation accuracy'
)